In [4]:
!pip install transformers
!pip install datasets
import pandas as pd
import requests
import os
import re
import csv
from transformers import pipeline
import torch.nn.functional as F
from transformers import AutoTokenizer , AutoModelForSequenceClassification
import torch
import numpy as np
import torch


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from transformers import Trainer
from transformers import TrainingArguments
#learning_rate = 5e-05
#batch_size = 16
#eval_batch_size = 4
seed = 40
#optimizer = Adam 
#with betas=(0.9,0.999) and epsilon=1e-08
adam_beta1 = 0.9
adam_beta2 =0.999
lr_scheduler_type = "linear"
num_epochs = 15
#args = TrainingArguments("test_trainer",report_to="wandb" ,logging_strategy = "epoch",evaluation_strategy="epoch",learning_rate = learning_rate,num_train_epochs = num_epochs,lr_scheduler_type =lr_scheduler_type, adam_beta1 = adam_beta1,adam_beta2 =adam_beta2  )


def run_trainer(model_name,out_dir,epoch,batch_size,df_train,df_valid,eval_bool):
  df_train = df_train[['text','label']].set_index('text')
  df_valid = df_valid[['text','label']].set_index('text')
  #display(df_train)
  #display(df_valid)

  dataset = ds.dataset(pa.Table.from_pandas(df_valid).to_batches())
  ### convert to Huggingface dataset
  validation_dataset_torch = Dataset(pa.Table.from_pandas(df_valid))

  dataset = ds.dataset(pa.Table.from_pandas(df_train).to_batches())
  ### convert to Huggingface dataset
  training_dataset_torch = Dataset(pa.Table.from_pandas(df_train))

  #print(training_dataset_torch)
  #print(validation_dataset_torch)

  from transformers import AutoTokenizer

  tokenizer = AutoTokenizer.from_pretrained(model_name,model_max_length=512)



  def tokenize_function(data):
      return tokenizer(data['text'], padding="max_length", truncation=True,)


  train_dataset = training_dataset_torch.map(tokenize_function, batched=True)
  eval_dataset = validation_dataset_torch.map(tokenize_function, batched=True)
  print(train_dataset)
  print(eval_dataset)

  import numpy as np
  from datasets import load_metric

  metric = load_metric("matthews_correlation")

  def compute_metrics(eval_pred):
      logits, labels = eval_pred
      predictions = np.argmax(logits, axis=-1)
      return metric.compute(predictions=predictions, references=labels)


  from transformers import AutoModelForSequenceClassification

  model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
  #model.to(device)
  args = TrainingArguments(
      #'/content/drive/MyDrive/fyp/fyp2/model/model2-supervised/' + f"{model_name}-finetuned-",
      #'/content/' + f"{model_name}-finetuned-",
      out_dir,
      #report_to="wandb",
      overwrite_output_dir = True,
      logging_dir = out_dir,
      evaluation_strategy = "epoch",
      logging_strategy = "epoch",
      #save_strategy = "NO",
      learning_rate=2.6510704963161386e-06,
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      num_train_epochs=epoch,
      weight_decay=0.01,
      metric_for_best_model='matthews_correlation',
      save_total_limit = 2,
      save_strategy = "epoch",
      load_best_model_at_end = True,
      seed = 18
      #push_to_hub=True,
  )



  trainer = Trainer(
      model,
      args,
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
      tokenizer=tokenizer,
      compute_metrics=compute_metrics
  )

  #trainer = Trainer(
    # model=model,
      #args=args,
    # train_dataset=train_dataset,
    #  eval_dataset=eval_dataset,
  #   compute_metrics=compute_metrics,
  #)

  
  if eval_bool:
    eval = trainer.evaluate()
    #print("bob")
    #print(eval)
    
    # training
  else:
    train_result = trainer.train() 
    logs = trainer.state.log_history
    # compute train results
    metrics = train_result.metrics
    max_train_samples = len(train_dataset)
    metrics["train_samples"] = min(max_train_samples, len(train_dataset))

    # save train results
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)

    # compute evaluation results
    metrics = trainer.evaluate()
    max_val_samples = len(eval_dataset)
    metrics["eval_samples"] = min(max_val_samples, len(eval_dataset))

    # save evaluation results
    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)
    import pickle
    save_logs_dir = out_dir
    name_logs = "logs"
    
    my_file = save_logs_dir + "/" + name_logs + ".p"
    print("saving to ",my_file )

    pickle.dump( logs, open( my_file, "wb" )    ) 

    ld = pickle.load( open( my_file, "rb" ) )


    print(logs)
    print(logs[2])
    print(type(logs))

    print(ld)
    print(type(ld))
    print(logs == ld)
    eval = trainer.evaluate()
    
  import torch
  torch.cuda.is_available() 
  from numba import cuda
  del trainer,args
  torch.cuda.empty_cache()
  #trainer.evaluate()
  return eval

